In [ ]:
pip install nibabel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install itk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 70.6 MB 1.1 MB/s 
     |████████████████████████████████| 20.3 MB 40.1 MB/s 
     |████████████████████████████████| 16.6 MB 33.0 MB/s 
     |████████████████████████████████| 15.0 MB 30.0 MB/s 
     |████████████████████████████████| 95.3 MB 59 kB/s 
     |████████████████████████████████| 54.5 MB 56 kB/s 


In [ ]:
pip install patchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nibabel as nib
import itk
#import SimpleITK as sitk
import glob
import os
import re
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, IntSlider, ToggleButton

#import tensorflow_datasets as tfds
from nibabel import ecat

from skimage import io
from patchify import patchify, unpatchify
import nibabel.processing
import skimage.transform as skTrans
import numpy as np
from matplotlib import pyplot as plt
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-255d6984-5029-197a-9164-bc2dec8ab3f9)


In [ ]:
df = pd.DataFrame(list(zip(path_ct, path_pet, path_mask)), columns =['CT_scan', 'PET', 'Mask'])

In [ ]:
half = int(len(df)*0.5)
half_df = df.iloc[:half, :]

In [ ]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, df, X_col = 'input_path', X2_col = 'input_path2', y_col = 'output_path',
                 batch_size = 2,
                 input_size = (224, 224, 3),
                 output_size = (224, 224, 3),
                 shuffle=False):
        
        self.df = df.copy()
        self.batch_size = batch_size
        self.input_size = input_size
        self.output_size = output_size
        self.col_x = X_col
        self.col_y = y_col
        self.col_x2 = X2_col
        self.shuffle = shuffle
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path, data_size):
        img_scan = np.array(nib.load(str(path)).get_fdata())
        patches = patchify(img_scan, (64, 64, 64), step=1)
        assert patches.shape 
        reconstructed_image = unpatchify(patches, img_scan.shape)
        result_img = skTrans.resize(reconstructed_image, (64,64,64), order=1, preserve_range=True)
        result_img = result_img/result_img.max()
        result_img = np.stack((result_img,)*3, axis=-1)
        #result_img = np.expand_dims(result_img, axis=-1)
        # patches = np.squeeze(patches)
        # for i in range(patches.shape[0]):
        #   for j in range(patches.shape[1]):
        #     single_patch_img = patches[i,j,:,:]
        # single_patch = np.expand_dims(single_patch_img, axis=-1) #np.stack((single_patch_img,)*3, axis=-1)
        # single_patch = np.concatenate((single_patch[0], single_patch[2], single_patch[2], single_patch[3]), axis=0)
        #ct_scan =  np.stack((ct_scan,)*3, axis=-1)

        # pet_img = np.array(nib.load(str(path2)).get_fdata())
        # pet_img = pet_img/pet_img.max()
        # pet_img = patchify(pet_img, (64, 64, 64), step=64)
        # pet_img = np.reshape(pet_img, (-1, pet_img.shape[3], pet_img.shape[4], pet_img.shape[5]))
        #result_img =  np.stack((result_img,)*3, axis=-1)
        
        #PET_CT_array = [np.array(ct_scan).astype(np.float32), np.array(pet_img).astype(np.float32)]
        return result_img
    
    def __get_output(self, path, data_size):
        mask_scan = np.array(nib.load(str(path)).get_fdata())
        patches = patchify(mask_scan, (64, 64, 64), step=1)
        assert patches.shape 
        reconstructed_image = unpatchify(patches, mask_scan.shape)
        result_mask = skTrans.resize(reconstructed_image, (64,64,64), order=1, preserve_range=True)
        result_mask = result_mask/result_mask.max()
        result_mask = np.stack((result_mask,)*3, axis=-1)
        #result_mask = result_mask/result_mask.max()
        #result_mask = np.expand_dims(result_mask, axis=-1)
        #patches = np.squeeze(patches)
        # for i in range(patches.shape[0]):
        #   for j in range(patches.shape[1]):
        #     single_patch_img = patches[i,j,:,:]
        # mask_patch = np.expand_dims(single_patch_img, axis=-1) #np.stack((single_patch_img,)*3, axis=-1)
        # mask_patch = np.concatenate((mask_patch[0], mask_patch[2], mask_patch[2], mask_patch[3]), axis=0)
        #result_mask =  np.stack((result_mask,)*3, axis=0)
        
        #mask = mask).astype(np.float32)

        return result_mask

    def __get_data(self, batches):
        # Generates data containing batch_size samples

        input_path_batch_1 = batches[self.col_x]
        input_path_batch_2 = batches[self.col_x2]
        output_path_batch = batches[self.col_y]
        X_batch_1 =[self.__get_input(x1, self.input_size) for x1 in input_path_batch_1]
        X_batch_2 = [self.__get_input(x2, self.input_size) for x2 in input_path_batch_2]
        Y_batch = [self.__get_output(y, self.output_size) for y in output_path_batch]

        return X_batch_1, X_batch_2,  Y_batch
    
    def __getitem__(self, index):
        
        X = []
        mask = []
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X1, X2, Y = self.__get_data(batches)
        
        #PET_CT_array = np.concatenate((X1, X2), axis=0)
        train_img = []
        both_images = [X1, X2]
        train_img.append(both_images)
        X_img = np.array(train_img).reshape(-1, 64, 64, 64, 3)

        mask_input = []
        mask_both = [Y, Y]
        mask_input.append(mask_both)
        Y_mask = np.array(mask_input).reshape(-1, 64, 64, 64, 3)

                
        return X_img, Y_mask
    
    def __len__(self):
        return self.n // self.batch_size


In [ ]:
#half_df

In [ ]:
train_size = int(len(half_df)*0.8)
val_size = int(len(half_df)*0.15)
test_size = int(len(half_df)*0.5)

In [ ]:
half_df = half_df.sample(frac=1)
df_train = half_df.iloc[:train_size,:]
df_val = half_df.iloc[train_size:train_size+val_size,:]

In [ ]:
batch_size = 2
traingen = CustomDataGen(df_train,
                         X_col='CT_scan',
                         X2_col='PET',
                         y_col='Mask',
                         batch_size=batch_size
                         )

valgen = CustomDataGen(df_val,
                         X_col='CT_scan',
                         X2_col='PET',
                         y_col='Mask',
                         batch_size=batch_size
                         )

In [ ]:
X, y = next(iter(traingen))

In [ ]:
X.shape

(4, 64, 64, 64, 3)

In [ ]:
y.shape

(4, 64, 64, 64, 3)

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 420 MB of archives.
After this operation, 3,369 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.4.1.50-1+cuda11.6 [420 MB]
Fetched 420 MB in 7s (59.9 MB/s)
(Reading database ... 155569 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155547 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.4.1.50-1+cuda11.6_amd64.deb ...
Unpacking libcudnn8 (8.4.1.50-1+c

In [ ]:
def conv_block(input, num_filters):
    x = Conv3D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv3D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPooling3D((2, 2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = Conv3DTranspose(num_filters, (2, 2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def Unet_encoder(inputs):
  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  b1 = conv_block(p4, 1024)

  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  return d4


def generate_input_layers(inputs_shape, num_inputs):
        inputs = []
        for i in range(num_inputs):
            inputs.append(Input((inputs_shape)))
        x = concatenate(inputs)
        return inputs, x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    #input_layer, concat_layer = generate_input_layers(input_shape, 2)
    input_layer = Input((input_shape))
    # d1 = Unet_encoder(inputs_1)
    # d2 = Unet_encoder(inputs_2)

    # d4 = Concatenate(axis=-1)([d1, d2])

    s1, p1 = encoder_block(input_layer, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv3D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(input_layer, outputs, name="U-Net")
    return model

In [ ]:
# Loss Function and coefficients to be used during training:
# def dice_coefficient(y_true, y_pred):
#     smoothing_factor = 1
#     flat_y_true = K.flatten(y_true)
#     flat_y_pred = K.flatten(y_pred)
#     return (2. * K.sum(flat_y_true * flat_y_pred) + smoothing_factor) / (K.sum(flat_y_true) + K.sum(flat_y_pred) + smoothing_factor)
import torch
def dice_coefficient(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    return K.mean((2. * intersection + smooth) / (union + smooth), axis=0)

def dice_coefficient_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def dice_metric(inputs, target):
  intersection = 2.0 * (target * inputs).sum()
  union = target.sum() + input.sum()
  if target.sum() == 0 and input.sum() == 0:
    return 1.0

  return intersection/union

def dice_loss(inputs, target):
  num = target.size(0)
  inputs = inputs.reshape(num, -1)
  target = target.reshape(num, -1)
  smooth = 1.0
  intersection = (inputs * target)
  dice = (2 * intersection.sum(1) + smooth)/ (inputs.sum(1) + target.sum(1) + smooth)
  dice = 1 + dice.sum()/num
  return dice


def bce_dice_loss(inputs, target):
  dicescore = dice_loss(inputs, target)
  bcescore = torch.nn.BCELoss()
  bceloss = bcescore(inputs, target)

  return bceloss + dicescore

In [ ]:
# def convolution3D(input, filters):
#   layer1 = Conv3D(filters=filters, kernel_size=(3,3,3), padding="same")(input)
#   layer2 =  tf.keras.layers.LeakyReLU()(layer1)

#   layer3 = Conv3D(filters=filters, kernel_size=(3,3,3), padding="same")(layer2)
#   layer4 =  tf.keras.layers.LeakyReLU()(layer3)
#   layer5 = tf.keras.layers.SpatialDropout3D(0.2)(layer4)

#   return layer5

# def pooling3D(input):
#   pool_layer = MaxPooling3D((2, 2, 2), padding="same")(input)

#   return pool_layer

# def Upsampling(input):
#   Upsample_layer = tf.keras.layers.UpSampling3D((2,2,2))(input)

#   return Upsample_layer

# def concatenate(upconv_input, forward_input):
#   concatenate_layer = Concatenate()([upconv_input, forward_input])
  
#   return concatenate_layer

# def output_convolution(input, num_classes, activation):
#   output_layer = tf.keras.layers.Conv3D(filters=num_classes, kernel_size=1, activation=activation, padding="same")(input)
  
#   return output_layer

# def encoder(input, filters):
#   layer1 = convolution3D(input, filters)
#   layer2 = pooling3D(layer1)
#   layer3 = tf.keras.layers.BatchNormalization()(layer2)
  
#   return layer1, layer3

# def bottleneck(input, filters):
#   layer1 = convolution3D(input, filters)
#   layer2 = tf.keras.layers.BatchNormalization()(layer1)
#   return layer2

# def decoder(upconv_input, forward_input, num_filters):
#   upconv_input = Upsampling(upconv_input)
#   layer1 = concatenate(upconv_input, forward_input)
#   layer2 = convolution3D(layer1, num_filters)
#   layer3 = tf.keras.layers.BatchNormalization()(layer2)
  
#   return layer3

# def UNET_3D(image_shape, n_classes):
#   input = Input((image_shape))

#   x = input
#   forwards = []
#   filters = (64, 128, 256, 512, 1024)

#   # Encoder
#   for i in range(len(filters) - 1):
#     forward, x = encoder(x, filters[i])
#     forwards.append(forward)
  
#   # Bottleneck
#   x = bottleneck(x, filters[-1])

#   # Decoder
#   for i in reversed(range(len(filters) - 1)):
#     x = decoder(x, forwards[i], filters[i])


#   if n_classes == 1:
#     activation = 'sigmoid'
#   else:
#     activation = 'softmax'
#   # Output layer
#   logits = output_convolution(x, n_classes, activation)
#   #output = Activation('sigmoid')(logits)
#   model = tf.keras.models.Model(input, logits, name='UNet3D')

#   return model


In [ ]:
n_classes = 1 
#0.001
LR = 0.001 
optim = tf.keras.optimizers.Adam(LR)
input = (64, 64, 64, 3)

#model = UNET_3D(input, n_classes=n_classes)
model = build_unet(input, n_classes=n_classes)

model.compile(optimizer = optim, loss=dice_coefficient_loss, metrics=[dice_coefficient])
print(model.summary())

In [ ]:
history=model.fit(traingen,
                  batch_size = 2, 
                  epochs=30,
                  validation_data = valgen)

Epoch 1/30
100/100 [==============================] - 3283s 33s/step - loss: 0.9949 - dice_coefficient: 0.0051 - val_loss: 0.9994 - val_dice_coefficient: 6.1916e-04
Epoch 2/30
100/100 [==============================] - 2996s 30s/step - loss: 0.9950 - dice_coefficient: 0.0050 - val_loss: 0.9892 - val_dice_coefficient: 0.0108
Epoch 3/30
100/100 [==============================] - 3191s 32s/step - loss: 0.9764 - dice_coefficient: 0.0236 - val_loss: 0.9763 - val_dice_coefficient: 0.0237
Epoch 4/30
100/100 [==============================] - 3217s 32s/step - loss: 0.9741 - dice_coefficient: 0.0259 - val_loss: 0.9691 - val_dice_coefficient: 0.0309
Epoch 5/30
100/100 [==============================] - 3047s 31s/step - loss: 0.9738 - dice_coefficient: 0.0262 - val_loss: 0.9676 - val_dice_coefficient: 0.0324
Epoch 6/30
100/100 [==============================] - 3218s 32s/step - loss: 0.9708 - dice_coefficient: 0.0292 - val_loss: 0.9988 - val_dice_coefficient: 0.0012
Epoch 7/30
100/100 [==========

KeyboardInterrupt: ignored

In [ ]:
model_save_path = '/content/drive/MyDrive/Model'
model.save(model_save_path+'Experiment3_2InputCT_PET_1MASK_3DImageSegmentation_3DUNET.h5')

In [ ]:
dice_coef = history.history['dice_coef']
val_coef = history.history['val_dice_coef']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(dice_coef))

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Loss')
plt.legend()
plt.figure()  

plt.plot(epochs, dice_coef, 'r', label='Training dice_coef')
plt.plot(epochs, val_coef, 'b', label='Validation dice_coef')
plt.title('Dice Coefficient')
plt.legend()

plt.show()

NameError: ignored